# Problem Statement (Final model)

* The objective of this project is to design and implement a vision-based Advanced Driver Assistance System (ADAS) using a front-facing driving video. The system must operate on continuous video input and perform the following tasks in real time:\

* Lane Detection and Lane Area Estimation\
The system shall detect left and right road lane boundaries using classical image processing techniques. Based on the detected lanes, the drivable lane area and lane center must be estimated and visualized on the video frames.

* Vehicle Detection using YOLO\
The system shall detect road vehicles such as cars and trucks using a pretrained YOLO object detection model. For each detected vehicle, a bounding box and confidence score must be displayed on the frame.

* Masked Region of Interest (ROI) Processing\
A fixed road-facing Region of Interest (ROI) must be defined ahead of the vehicle. Binary masking shall be applied to isolate this region, and only vehicles entering this masked region shall be considered for further safety and traffic analysis.

* Vehicle Counting within ROI\
The system shall count the total number of unique vehicles that enter the masked ROI during the video. Each vehicle must be counted only once, based on its spatial entry into the region.

* Forward Collision Risk Estimation\
The system shall estimate collision risk using the relative position and size of detected vehicle bounding boxes within the ROI. Vehicles closer to the bottom of the frame or exceeding a predefined size threshold shall trigger appropriate safety warnings.

* Lane Departure Warning\
The system shall monitor the vehicle’s position relative to the estimated lane center. If the deviation exceeds a defined threshold, a lane departure warning must be generated.

* Unified ADAS Visualization\
All outputs including lane overlays, detected vehicles, ROI mask, vehicle count, and warning messages must be displayed together on a single annotated video stream.

   



In [12]:
!pip install ultralytics opencv-python
import cv2
import numpy as np
from ultralytics import YOLO
# pip installing and importing all required libraries

In [13]:
# Load YOLO model
model = YOLO("yolov8n.pt")

# Load video
video_path = "/content/01095.mp4"
cap = cv2.VideoCapture(video_path)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Frame Width
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #  Frame Height
fps    = int(cap.get(cv2.CAP_PROP_FPS))          # Frames per second


# ROI Mask



In [14]:
def create_roi_mask(frame):

    mask = np.zeros((height, width), dtype=np.uint8) # Initialize the mask

    # Create a polygonal mask( Trapezoid )
    points = np.array([
        [width*0.1, height],
        [width*0.45, height*0.6],
        [width*0.55, height*0.6],
        [width*0.9, height]
    ], dtype=np.int32)

    cv2.fillPoly(mask, [points], 255)
    return mask, points


# Lane Detection

In [15]:
def detect_lanes(frame, mask):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Grayscale version of the video frame

    blur = cv2.GaussianBlur(gray, (5,5), 0) # Blurring the frame

    edges = cv2.Canny(blur, 50, 150) # Applying edge detection

    masked = cv2.bitwise_and(edges, mask) # Apply the mask over edges

    lines = cv2.HoughLinesP(masked, 1, np.pi/180, 100, minLineLength=50, maxLineGap=150) # Line Detection

    left = []
    right = []

    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            slope = (y2-y1)/(x2-x1+1e-15) # Add 0.001 to avoid division by zero

            # Distinguishing into right and left lanes based on slope (a certain threshold)
            if slope < -0.5:
                left.append(line[0])
            elif slope > 0.5:
                right.append(line[0])

    return left, right


# Lane Area and Lane Center

We need this function because we need to decide the extent(spread) of the lane about the center of the lane

In [16]:
def draw_lanes(frame, left, right):

    if len(left) == 0 or len(right) == 0:
        return None, None, None # Return None for all three values if no lanes

    left_avg = np.mean(left, axis=0).astype(int)
    right_avg = np.mean(right, axis=0).astype(int)

    # Polygonal Region(Quadilateral)
    # Bottom Left, Top Left, Bottom Right , Top Right point coordinates
    lane_area = np.array([
        [left_avg[0], left_avg[1]],
        [left_avg[2], left_avg[3]],
        [right_avg[2], right_avg[3]],
        [right_avg[0], right_avg[1]]
    ])

    # Polygonal shape
    overlay = frame.copy()
    cv2.fillPoly(overlay, [lane_area], (0,255,0))

    # Brightness and Contrastness adjustment
    cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)

    """ Ceneter of lane(only Y needed as X coordinate is zero as refernce is the
     line parallel to lanes )"""

    lane_center = (left_avg[2] + right_avg[2]) // 2

    return left_avg, right_avg, lane_center

# Vehicle Detection Function

In [17]:
def detect_vehicles(frame):
    results = model(frame)[0]
    vehicles = []

    for box in results.boxes:
        cls = int(box.cls[0])
        label = model.names[cls]

        # Creating labels for objects like car,bus,etc.
        if label in ["car", "truck", "bus", "motorcycle"]:

            # Conversion to intger type list from tensor
            x1,y1,x2,y2 = box.xyxy[0].int().tolist()

            conf = float(box.conf[0]) # Confidence score

            vehicles.append((x1,y1,x2,y2,conf,label)) # Add vehicles' attributes to the list

    return vehicles


# Count Vehicles

In [18]:
vehicle_count = 0
counted_points = []

def count_vehicles(vehicles, mask):
    global vehicle_count

    for v in vehicles:
        x1,y1,x2,y2,_,_ = v # We do not require label and confidence scores of vehicles

        # Center
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2


        if mask[cy, cx] == 255:
            point = (cx//20, cy//20)

            if point not in counted_points:

              #Ensuring vehicles are being counted only once
                counted_points.append(point)
                vehicle_count += 1


# Lane Departutre

In [19]:
def check_lane_departure(left_lane_avg, right_lane_avg, frame_width):

    # If either averaged lane is None (meaning no lines were detected and averaged)
    if left_lane_avg is None or right_lane_avg is None:
        return 'unknown', 0, None, None

    vehicle_center = frame_width // 2

    # Extract x-coordinates from the averaged lines. These are already single arrays.
    # Based on draw_lanes, the 'lane_center' is derived from left_avg[2] and right_avg[2],
    # which correspond to the x2 (bottom x) coordinate of the averaged line segments.
    left_x = left_lane_avg[2]
    right_x = right_lane_avg[2]

    lane_center = (left_x + right_x) // 2
    deviation = abs(vehicle_center - lane_center)


    # Check for Left and Right Direction

    if vehicle_center < lane_center:
        direction = 'left'
    else:
        direction = 'right'

    warning_threshold = frame_width * 0.15
    critical_threshold = frame_width * 0.20

    if deviation >= critical_threshold:
        message = f"LANE DEPARTURE! DRIFTING {direction.upper()}"
        return 'critical', deviation, direction, message

    elif deviation >= warning_threshold:
        message = f"Lane Departure: Drifting {direction}"
        return 'warning', deviation, direction, message

    else:
        return 'safe', deviation, direction, None

# Check Collision

In [20]:
def assess_collision_risk(vehicles, frame_height):

    for v in vehicles:

        x1,y1,x2,y2,_,_ = v # We do not need confidence score and label here
        box_height = y2 - y1

  # Criteria for Collision (ratio factors for collisions ) seems to be optimal

        bottom_y = y2
        area = (x2 - x1) * (y2 - y1)

        critical_y = frame_height * 0.85
        warning_y = frame_height * 0.80
        critical_area = (frame_height * 1280) * 0.20
        warning_area = (frame_height * 1280) * 0.15

        if bottom_y >= critical_y or area >= critical_area:
          # Display if satisy above condition
            return 'critical', [{'vehicle': v, 'risk': 'critical'}]

        elif bottom_y >= warning_y or area >= warning_area:
            return 'warning', [{'vehicle': v, 'risk': 'warning'}]

    return 'safe', []


# Output

In [21]:
import time

# Video Writer
out = cv2.VideoWriter(
    "ADAS_Output.avi",
    cv2.VideoWriter_fourcc(*"XVID"),
    fps,
    (width, height)
)

frame_count = 0

while True:

    start_time = time.time()

    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    height, width = frame.shape[:2]

    # ROI and points in ROI
    mask, roi_points = create_roi_mask(frame)

    # Left, Right and Center of the lane
    left, right = detect_lanes(frame, mask)
    # Unpack the averaged left and right lines along with the lane center
    left_avg_line, right_avg_line, lane_center = draw_lanes(frame, left, right)

    # Detecting and Counting Vehicles
    vehicles = detect_vehicles(frame)
    count_vehicles(vehicles, mask)
    vehicle_count = len(vehicles)

    # Collision Risk
    collision_status, risky_vehicles = assess_collision_risk(vehicles, height)

    # Lane Departure
    # Pass the averaged lane lines to check_lane_departure
    lane_status, deviation, direction, lane_message = check_lane_departure(left_avg_line, right_avg_line, width)

    # Draw ROI
    cv2.polylines(frame, [roi_points], True, (255,0,0), 2)

    # Draw Vehicles
    for v in vehicles:

        x1, y1, x2, y2, conf, label = v

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

        text = f"{label}: {conf:.2f}"

        (tw, th), _ = cv2.getTextSize(
            text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
        )

        cv2.rectangle(
            frame,
            (x1, y1 - th - 6),
            (x1 + tw + 4, y1),
            (0, 0, 255),
            -1
        )

        cv2.putText(
            frame,
            text,
            (x1 + 2, y1 - 4),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            1
        )

    # FPS
    fps_display = 1.0 / (time.time() - start_time)

    # Annotate
    cv2.putText(frame, f"FPS: {fps_display:.2f}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)

    cv2.putText(frame, f"Width: {width}", (20, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)

    cv2.putText(frame, f"Height: {height}", (20, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)

    cv2.putText(frame, f"Frame: {frame_count}", (20, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)

    cv2.putText(frame, f"Vehicles: {vehicle_count}", (20, 160),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0), 2)



    # Collision warning
    if collision_status == 'critical':
        cv2.putText(frame, "COLLISION WARNING!",
                    (width//3, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

    elif collision_status == 'warning':
        cv2.putText(frame, "CAUTION: Vehicle Close",
                    (width//3, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,165,255), 3)

    # Lane departure warning with drifting direction
    if lane_status == 'critical' and lane_message is not None:
        cv2.putText(frame, lane_message,
                    (width//3, 120), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)

    elif lane_status == 'warning' and lane_message is not None:
        cv2.putText(frame, lane_message,
                    (width//3, 120), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,255), 3)

    # Write frame to output video
    out.write(frame)

# Save the video
cap.release()
out.release()
print("Video saved successfully")



0: 384x640 1 car, 1 truck, 137.1ms
Speed: 2.5ms preprocess, 137.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 171.8ms
Speed: 2.2ms preprocess, 171.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 184.4ms
Speed: 2.0ms preprocess, 184.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 127.8ms
Speed: 1.9ms preprocess, 127.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 224.2ms
Speed: 4.4ms preprocess, 224.2ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 269.4ms
Speed: 1.8ms preprocess, 269.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 147.9ms
Speed: 1.8ms preprocess, 147.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 134.1ms
Speed: 2.9ms preproces

In [22]:
!ffmpeg -y -i ADAS_Output.avi -vcodec libx264 ADAS_Output.mp4
from IPython.display import Video

Video("ADAS_Output.mp4", embed=True)


Output hidden; open in https://colab.research.google.com to view.